In [4]:
!pip install tensorflow==1.15.3

     |████████████████████████████████| 110.5MB 7.1MB/s eta 0:00:01
     |████████████████████████████████| 51kB 5.2MB/s eta 0:00:011
     |████████████████████████████████| 1.3MB 4.5MB/s eta 0:00:01
     |████████████████████████████████| 512kB 2.5MB/s eta 0:00:01
     |████████████████████████████████| 71kB 1.4MB/s eta 0:00:01
     |████████████████████████████████| 133kB 4.8MB/s eta 0:00:01
     |████████████████████████████████| 51kB 2.9MB/s eta 0:00:01
     |████████████████████████████████| 61kB 3.1MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 841kB/s eta 0:00:01
     |████████████████████████████████| 3.3MB 1.6MB/s eta 0:00:01
     |████████████████████████████████| 92kB 6.5MB/s eta 0:00:011
  Created wheel for termcolor: filename=termcolor-1.1.0-cp37-none-any.whl size=4832 sha256=04a0e213075ed7e10371a75452585965bc128ec3501eb3027fff04c16d89bca6
  Stored in directory: /home/vvs/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Create

In [5]:
import tensorflow as tf
tf.__version__

'1.15.3'

In [5]:
import tensorflow as tf
import numpy as np
import os, random, time
import compress_fasttext
from Model import Model
from utils import load_data, build_vocab, preview_data

if "CUDA_VISIBLE_DEVICES" not in os.environ: 
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
FLAGS = tf.flags.FLAGS

tf.flags.DEFINE_boolean('train1', True, 'train model')
tf.flags.DEFINE_integer('display_interval', 500, 'step interval to display information')
tf.flags.DEFINE_boolean('show_predictions', False, 'show predictions in the test stage')
tf.flags.DEFINE_string('word_vector', '/home/vvs/Downloads/d_parse/models/ft_freqprune_100K_20K_pq_100.bin', 'word vector')
tf.flags.DEFINE_string('prefix', 'dev', 'prefix for storing model and log')
tf.flags.DEFINE_integer('vocab_size', 1000, 'vocabulary size')
tf.flags.DEFINE_integer('max_edu_dist', 20, 'maximum distance between two related edus') 
tf.flags.DEFINE_integer('dim_embed_word', 300, 'dimension of word embedding')
tf.flags.DEFINE_integer('dim_embed_relation', 300, 'dimension of relation embedding')
tf.flags.DEFINE_integer('dim_feature_bi', 4, 'dimension of binary features')
tf.flags.DEFINE_boolean('use_structured', True, 'use structured encoder')
tf.flags.DEFINE_boolean('use_speaker_attn', True, 'use speaker highlighting mechanism')
tf.flags.DEFINE_boolean('use_shared_encoders', False, 'use shared encoders')
tf.flags.DEFINE_boolean('use_random_structured', False, 'use random structured repr.')
tf.flags.DEFINE_integer('num_epochs', 50, 'number of epochs')
tf.flags.DEFINE_integer('num_units', 256, 'number of hidden units')
tf.flags.DEFINE_integer('num_layers', 1, 'number of RNN layers in encoders')
tf.flags.DEFINE_integer('num_relations', 16, 'number of relation types')
tf.flags.DEFINE_integer('batch_size', 4, 'batch size')
tf.flags.DEFINE_float('keep_prob', 0.5, 'probability to keep units in dropout')
tf.flags.DEFINE_float('learning_rate', 0.1, 'learning rate')
tf.flags.DEFINE_float('learning_rate_decay', 0.98, 'learning rate decay factor')
    
def get_summary_sum(s, length):
    loss_bi, loss_multi = s[0] / length, s[1] / length
    prec_bi, recall_bi = s[4] * 1. / s[3], s[4] * 1. / s[2]
    f1_bi = 2 * prec_bi * recall_bi / (prec_bi + recall_bi)
    prec_multi, recall_multi = s[5] * 1. / s[3], s[5] * 1. / s[2]
    f1_multi = 2 * prec_multi * recall_multi / (prec_multi + recall_multi)
    return [loss_bi, loss_multi, f1_bi, f1_multi]    
    
map_relations = {}
data_train = load_data('/home/vvs/Downloads/d_parse/chinadataset/train_spect_ru.json', map_relations)
data_test = load_data('/home/vvs/Downloads/d_parse/chinadataset/TEST_spect_ru.json', map_relations)
valid_size = int(FLAGS.valid_ratio * len(data_train))
data_valid = data_train[-valid_size:]
data_train = data_train[:-valid_size]
vocab, embed = build_vocab(data_train)
print("Dataset sizes: %d/%d/%d" % (len(data_train), len(data_test), len(data_valid)))

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
with sess.as_default():
    model = Model(sess, FLAGS, embed, data_train)

    global_step = tf.Variable(0, name="global_step", trainable=False)
    global_step_inc_op = global_step.assign(global_step + 1)    
    epoch = tf.Variable(0, name="epoch", trainable=False)
    epoch_inc_op = epoch.assign(epoch + 1)

    saver = tf.train.Saver(
        write_version=tf.train.SaverDef.V2,
        max_to_keep=None, 
        pad_step_number=True, 
        keep_checkpoint_every_n_hours=1.0
    )        
    
    summary_list = [
        "loss_bi", "loss_multi",
        "prec_bi", "recall_bi", "f1_bi",
        "prec_multi", "recall_multi", "f1_multi"
    ]
    summary_num = len(summary_list)
    len_output_feed = 6

    if FLAGS.is_train:
        if tf.train.get_checkpoint_state(FLAGS.model_dir):
            print("Reading model parameters from %s" % FLAGS.model_dir)
            saver.restore(sess, tf.train.latest_checkpoint(FLAGS.model_dir))
        else:
            print("Created model with fresh parameters")
            sess.run(tf.global_variables_initializer())  
            model.initialize(vocab)          
            
        print("Trainable variables:")
        for var in tf.trainable_variables():
            print(var)

        train_writer = tf.summary.FileWriter(FLAGS.log_dir+'/'+ "train")
        valid_writer = tf.summary.FileWriter(FLAGS.log_dir+'/'+"valid")
        test_writer = tf.summary.FileWriter(FLAGS.log_dir+'/'+"test")
        summary_placeholders = [tf.placeholder(tf.float32) for i in range(summary_num)]
        summary_op = [tf.summary.scalar(summary_list[i], summary_placeholders[i]) for i in range(summary_num)]
        
        train_batches = get_batches(data_train, FLAGS.batch_size)
        valid_batches = get_batches(data_valid, FLAGS.batch_size)
        test_batches = get_batches(data_test, FLAGS.batch_size)
        
        best_test_f1 = [0] * 2
        while True:
            epoch_inc_op.eval()
            summary_steps = 0
            
            random.shuffle(train_batches)
            start_time = time.time()
            s = np.zeros(len_output_feed)
            
            for batch in train_batches:
                ops = model.step(batch, is_train=True)
                    
                for i in range(len_output_feed):
                    s[i] += ops[i]
                        
                summary_steps += 1
                global_step_inc_op.eval()
                global_step_val = global_step.eval()         
                if global_step_val % FLAGS.display_interval == 0:
                    print("epoch %d, global step %d (%.4fs/step):" % (
                        epoch.eval(), global_step_val, 
                        (time.time() - start_time) * 1. / summary_steps
                    ))
                    summary_sum = get_summary_sum(s, summary_steps)
                    for k in range(summary_num):
                        print("  train %s: %.5lf" % (
                            summary_list[k], 
                            summary_sum[k]
                        ))
                        
                    print("  best test f1:", best_test_f1[0], best_test_f1[1])
                        
            summary_sum = get_summary_sum(s, len(train_batches))            
            summaries = sess.run(summary_op, feed_dict=dict(list(zip(summary_placeholders, summary_sum))))
            for s in summaries:
                train_writer.add_summary(summary=s, global_step=epoch.eval())                        
            print("epoch %d (learning rate %.5lf)" % \
                (epoch.eval(), model.learning_rate.eval()))
            for k in range(summary_num):
                print("  train %s: %.5lf" % (summary_list[k], summary_sum[k]))                   
            
            s = np.zeros(len_output_feed)
            for batch in valid_batches:
                ops = model.step(batch)
                for i in range(len_output_feed):
                    s[i] += ops[i]
            summary_sum = get_summary_sum(s, len(valid_batches))
            summaries = sess.run(summary_op, feed_dict=dict(list(zip(summary_placeholders, summary_sum))))
            for s in summaries:
                valid_writer.add_summary(summary=s, global_step=epoch.eval())
            for k in range(summary_num):
                print("  valid %s: %.5lf" % (summary_list[k], summary_sum[k]))                              
            
            s = np.zeros(len_output_feed)
            random.seed(0)
            for batch in test_batches:
                ops = model.step(batch)
                for i in range(len_output_feed):
                    s[i] += ops[i]
            summary_sum = get_summary_sum(s, len(test_batches))
            summaries = sess.run(summary_op, feed_dict=dict(list(zip(summary_placeholders, summary_sum))))
            for s in summaries:
                test_writer.add_summary(summary=s, global_step=epoch.eval())            
            for k in range(summary_num):
                print("  test %s: %.5lf" % (summary_list[k], summary_sum[k])) 
                
            if summary_sum[-1] > best_test_f1[1]:
                best_test_f1[0] = summary_sum[-4]
                best_test_f1[1] = summary_sum[-1]
            
            print("  best test f1:", best_test_f1[0], best_test_f1[1])
            
            model.learning_rate_decay_op.eval()             
            
            saver.save(sess, "%s/checkpoint" % FLAGS.model_dir, global_step=epoch.eval())                                            
    else:
        print("Reading model parameters from %s" % FLAGS.model_dir) 
        saver.restore(sess, tf.train.latest_checkpoint(FLAGS.model_dir))
        
        test_batches = get_batches(data_test, 1, sort=False)
    
        s = np.zeros(len_output_feed)
        random.seed(0)
        idx = 0
        cnt_golden, cnt_pred, cnt_cor_bi, cnt_cor_multi = [], [], [], []
        for k, batch in enumerate(test_batches):
            if len(batch[0]["edus"]) == 1: 
                continue
            
            ops = model.step(batch)
            for i in range(len_output_feed):
                s[i] += ops[i]
                
            if FLAGS.show_predictions:
                idx = preview_data(batch, ops[-1], map_relations, vocab, idx)
            
            cnt_golden.append(ops[2])    
            cnt_pred.append(ops[3])    
            cnt_cor_bi.append(ops[4])    
            cnt_cor_multi.append(ops[5])    
                
        summary_sum = get_summary_sum(s, len(test_batches))
        
        print(cnt_golden)
        print(cnt_pred)
        print(cnt_cor_bi)
        print(cnt_cor_multi)
        print("Test:")
        for k in range(summary_num):
            print("  test %s: %.5lf" % (summary_list[k], summary_sum[k]))   

DuplicateFlagError: The flag 'display_interval' is defined twice. First from /home/vvs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, Second from /home/vvs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py.  Description from first occurrence: step interval to display information